# WOMBAT Motor Control and Data Logging

## Motor Controller for WOMBAT Maze

**The cell below is a python executable for control over the motors.** 

#### Getting Started
    
* First, run the arduino code titled Motor_controller.ino in the Arduino IDE (verify + upload) before running the motor controller python script here. The associated Arduino code is also included at the bottom of this notebook if needed
* Next, run the motor controller python script below (shift + return)

#### Motor GUI
* The python code will generate a small gui under the cell that can be used to control the motors on the olfactometer
* The buttons work as on/off toggle switches. Run up to two motors simultaneously.
* This is used for priming/purging the reward lines or can be used for troubleshooting.
#### Priming
* When priming the lines run the liquid reward until a small amount has dispensed from the gavage
#### Purging
* When purging the lines, first remove the gavage from the nosepoke
* Run any remaining liquid reward out of the lines
* Next, run warm soapy water though the lines until the lines run clear 
* Last, let the lines run dry for 15 seconds
 #### Kill Serial Connection
* To exit the serial connection interrupt the kernel by either pressing the square button at the top of the notebook or finding the interrupt kernel command under Kernel
  
**Potential Issues:**
* If there is an issue with the port being busy for the arduino, restart the kernel and clear all outputs (under kernel at the top of the jupyter notebook) - this issue occurs when the python script is run prior to running the arduino script
* If there is an issue with the port working when trying to run the python script, double check that the port name is correct: 
> arduino = serial.Serial('correct port name'), 9600)


In [ ]:
# T Stowell 
# Python exectuable for motor controller

import serial
import ipywidgets as widgets
from IPython.display import display

arduino = serial.Serial('/dev/tty.usbmodem101', 9600)

def create_motor_toggle(motor_num, description):
    toggle = widgets.ToggleButton(
        description=description,
        layout=widgets.Layout(width='300px')
    )
    
    def on_toggle(change):
        cmd = f"S{motor_num}" if change['new'] else f"T{motor_num}"
        arduino.write(f"{cmd}\n".encode())
    
    toggle.observe(on_toggle, names='value')
    return toggle

# Create toggle buttons
motor_toggles = [
    (1, "Scent 1 Right"),
    (2, "Scent 2 Right"),
    (3, "Reward Right"),
    (4, "Scent 1 Left"),
    (5, "Scent 2 Left"),
    (6, "Reward Left")
]

for num, desc in motor_toggles:
    display(create_motor_toggle(num, f"M{num}: {desc}"))

def close_serial():
    arduino.close()

import atexit
atexit.register(close_serial)


## Data Logging for WOMBAT Maze

**The cell below contains the python code to log data for each trial of the maze.**

#### Data Logging
* First, run the arduino code WOMBAT.ino in the Arduino IDE (compile + run) before running the data logging python script here. The arduino code is in a markdown cell below if needed at the bottom of the notebook if needed
* Next, run the data logging python script below (shift + return)

#### Data Output
* Data from the trials will output to the folder "Wombat_logs" 
* Data from the trial will also output in realtime below the python executable

#### Kill the Serial Connection
* To exit the serial connection interrupt the kernel by either pressing the square button at the top of the notebook or finding the interrupt kernel command under Kernel


In [ ]:
import serial
import os
from datetime import datetime

# prompt user to enter animal ID
rat_id = input("Enter animal identification number: ")

# serial port 
ser = serial.Serial('/dev/cu.usbmodem101', 9600, timeout=1)

# captures the date for file logs
now = datetime.now()
date_str = now.strftime("%m-%d-%y")

# creates the folder path logs/rat_123/04-01-2025"
folder_path = os.path.join("WOMBAT_logs", "training_stage_1", f"rat_{rat_id}", date_str)

# creates the directory
os.makedirs(folder_path, exist_ok=True)

# creates the filename 
# e.g., "arduino_log_rat123_2025-03-25_153001.txt"
timestamp_str = now.strftime('%m-%d-%y_%H%M')
filename = f"{rat_id}_{timestamp_str}.txt"

# Combine folder path and filename into one full path
file_path = os.path.join(folder_path, filename)

print(f"Logging to {file_path} ...")

try:
    with open(file_path, 'w', buffering=1) as outfile:
        # (Optional) write headers
        outfile.write("WOMBAT Alternation Maze\n")
        outfile.write(now.strftime("%m-%d-%Y %H:%M:%S") + "\n")
        outfile.write("***********************************\n\n")
        outfile.write(f"**Animal Identification: {rat_id}\n\n")
        outfile.write("*************************************\n")
        outfile.write("**********Experiment Started**********\n\n")

        # Continuously read and log serial data
        while True:
            line = ser.readline().decode('utf-8', errors='replace').strip()
            if line:
                # Prepend timestamp to each line
                current_time = datetime.now().strftime("%H:%M:%S")
                formatted_line = f"[{current_time}] {line}"
                print(formatted_line)
                outfile.write(formatted_line + "\n")

except KeyboardInterrupt:
    print("\nInterrupted by user. Closing serial connection...")
    ser.close()


----
----

# Peripherals 

## Capactive Sensor Calibration

**Useful if you need to check the value coming from the capacitive sensors**
* Only requires running arduino code, which can either be found under wombat/Arduino_code/Cap_sensor_test.ino or copy and paste from the cell containing the code in the Ardiuno section of this notebook (below) 
* Open the serial monitor under tools in the IDE to see the outputs
* The thresholds for triggering the cap sensor can be changed by changing the values for the following
    * const float TOUCH_THRESHOLD_1 = 5.25;  
    * const float TOUCH_THRESHOLD_2 = 5.25;
* Note that sensor 1 (channel 0) corresponds to the right nosepoke and sensor 2 (channel 3) corresponds to the left nosepoke
* Also note that there is a shield pin that is used to control for noise in the system and it should be grounded to something separate from the main sensor

## Piezo Buzzer Calibration

**Useful to check that piezo buzzers are working and if you want to quickly test different frequency settings**

* Requires running the piezo buzzer arduino code and the python exectuable below
* Python executable will generate a simple on/off toggle switch
* Change the following line in the arduino code to test different frequencies:
    * const uint16_t TONE_HZ = 4000;     // where 4000 is 4000 Hz


In [ ]:
"""
Simple toggle button GUI that sends '1' or '0' over serial
to switch a buzzer on the 

Adjust SERIAL_PORT to whatever the arduino is using  
"""

import tkinter as tk
import serial
import serial.tools.list_ports
import sys

# ------- CONFIG -------------------------------------------------
SERIAL_PORT = '/dev/cu.usbmodem1201'      # something like "COM3" or "/dev/ttyACM0"
BAUD = 9600

def find_arduino_port():
    """Return the first port whose USB description looks like Arduino"""
    for p in serial.tools.list_ports.comports():
        if "Arduino" in p.description or "CDC" in p.description:
            return p.device
    return None

port = SERIAL_PORT or find_arduino_port()
if port is None:
    sys.exit("No Arduino-like serial port found – edit SERIAL_PORT manually!")

try:
    ser = serial.Serial(port, BAUD, timeout=1)
except serial.SerialException as e:
    sys.exit(f"Could not open serial port {port}: {e}")

# Make sure the board had time to reset on port open
ser.reset_input_buffer()

# ---------------- TK GUI --------------------
root = tk.Tk()
root.title("Buzzer toggle")

state = tk.BooleanVar(value=False)   # False = OFF, True = ON

def toggle():
    new_state = not state.get()
    state.set(new_state)
    ser.write(b'1' if new_state else b'0')
    button.configure(text="Buzzer ON" if new_state else "Buzzer OFF",
                     bg="#ff6666" if new_state else "#cccccc")

button = tk.Button(root,
                   text="Buzzer OFF",
                   width=20,
                   height=3,
                   bg="#cccccc",
                   command=toggle)

button.pack(padx=20, pady=20)

def on_close():
    ser.write(b'0')        # be sure to turn it off
    ser.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_close)
root.mainloop()


# Arduino Codes

### Motor Controller

```cpp

// T Stowell
// Motor controller
// Works in tandem with arduino uno microcontroller and python script (in jupyter notebook)

#include <Wire.h>
#include <Adafruit_MotorShield.h>

Adafruit_MotorShield AFMSLeft(0x60);
Adafruit_MotorShield AFMSRight(0x61);

Adafruit_DCMotor* motors[6] = {
  AFMSLeft.getMotor(1),  // M1: Scent 1 Right
  AFMSLeft.getMotor(2),  // M2: Scent 2 Right
  AFMSLeft.getMotor(3),  // M3: Reward Right
  AFMSRight.getMotor(1), // M4: Scent 1 Left
  AFMSRight.getMotor(2), // M5: Scent 2 Left
  AFMSRight.getMotor(3)  // M6: Reward Left
};

void setup() {
  Serial.begin(9600);
  AFMSLeft.begin();
  AFMSRight.begin();
  
  for(int i=0; i<6; i++) {
    motors[i]->setSpeed(150);
  }
}

void loop() {
  if(Serial.available() > 0) {
    String cmd = Serial.readStringUntil('\n');
    cmd.trim();
    
    if(cmd.length() == 2) {
      char action = cmd[0];
      int motorNum = cmd[1] - '1'; // Convert to 0-5 index
      
      if(motorNum >=0 && motorNum <6) {
        if(action == 'S') {
          motors[motorNum]->run(FORWARD);
        }
        else if(action == 'T') {
          motors[motorNum]->run(RELEASE);
        }
      }
    }
  }
}

```

### Capacitive Sensors

```cpp 

#include <Wire.h>
#include "FDC1004.h"

FDC1004 sensor;

// Sensor channels
const uint8_t SENSOR1_CHANNEL = 0;  // CIN1
const uint8_t SENSOR2_CHANNEL = 3;  // CIN2

const uint8_t CAPDAC = 0;

// Shield assignments (if using them physically)
const uint8_t SHIELD1 = 1;  // SHLD1
const uint8_t SHIELD2 = 2;  // SHLD2

// Adjust thresholds as needed
const float TOUCH_THRESHOLD_1 = 4.5;  
const float TOUCH_THRESHOLD_2 = 5.25;


// Track previous touch states
bool wasTouched1 = false;
bool wasTouched2 = false;

void setup() {
  Serial.begin(9600);
  Wire.begin();

  // Configure each sensor channel with its respective shield
  // If you need shields explicitly, see library's `configureMeasurement` function
  sensor.configureMeasurementSingle(0, SENSOR1_CHANNEL, CAPDAC); // Meas ID=0 → CIN1
  sensor.configureMeasurementSingle(1, SENSOR2_CHANNEL, CAPDAC); // Meas ID=1 → CIN2




  Serial.println("Touch sensors ready:");
}

void loop() {
  // --- SENSOR 1 ---
  uint16_t rawValue1;
  float capacitance1;

  if (sensor.measureChannel(SENSOR1_CHANNEL, CAPDAC, &rawValue1) == 0) {
    // Convert raw data to pF
    capacitance1 = ((int16_t)rawValue1) / 5242.88;
    bool isTouched1 = capacitance1 > TOUCH_THRESHOLD_1;

    // Detect state change for Sensor 1
    if (isTouched1 && !wasTouched1) {
      Serial.print("Sensor 1: Touch detected! pF = ");
      Serial.println(capacitance1, 2); // prints 2 decimal places
    } 
    else if (!isTouched1 && wasTouched1) {
      Serial.print("Sensor 1: Touch released! pF = ");
      Serial.println(capacitance1, 2);
    }
  }

  // --- SENSOR 2 ---
  uint16_t rawValue2;
  float capacitance2;

  if (sensor.measureChannel(SENSOR2_CHANNEL, CAPDAC, &rawValue2) == 0) {
    // Convert raw data to pF
    capacitance2 = ((int16_t)rawValue2) / 5242.88;
    bool isTouched2 = capacitance2 > TOUCH_THRESHOLD_2;

    // Detect state change for Sensor 2
    if (isTouched2 && !wasTouched2) {
      Serial.print("Sensor 2: Touch detected! pF = ");
      Serial.println(capacitance2, 2);
    } 
    else if (!isTouched2 && wasTouched2) {
      Serial.print("Sensor 2: Touch released! pF = ");
      Serial.println(capacitance2, 2);
    }
  }

  delay(100);  // adjust for responsiveness as needed
}



### Piezo Buzzer

```cpp


/* -------------------------------------------
   Buzzer-toggle demo
   -------------------------------------------
   Connect piezo buzzer (or an active buzzer module)
   – Signal  : D9   (change BUZZ_PIN if you like)
   – GND     : GND
   ------------------------------------------- */

const uint8_t BUZZ_PIN = 9;        // digital I/O pin for the buzzer
const uint16_t TONE_HZ = 4000;     // tone frequency if you use tone()
bool buzzerState = false;          // remembers current state

void setup() {
  pinMode(BUZZ_PIN, OUTPUT);
  digitalWrite(BUZZ_PIN, LOW);
  Serial.begin(9600);
  // Give the PC a moment to open the port:
  while (!Serial) { /* Leonardo/MKR ZERO need this to enumerate */ }
  Serial.println(F("Ready for commands: '1'=on  '0'=off"));
}

void loop() {
  if (Serial.available()) {
    char cmd = Serial.read();
    if (cmd == '1') {                 // turn ON
      buzzerState = true;
#ifdef __AVR__
      // AVR boards can use tone()
      tone(BUZZ_PIN, TONE_HZ);        // continuous tone
#else
      digitalWrite(BUZZ_PIN, HIGH);   // active buzzer module
#endif
      Serial.println(F("ON"));
    } else if (cmd == '0') {          // turn OFF
      buzzerState = false;
#ifdef __AVR__
      noTone(BUZZ_PIN);
#else
      digitalWrite(BUZZ_PIN, LOW);
#endif
      Serial.println(F("OFF"));
    }
  }

  // (nothing else happens; main loop idles at a few µs per pass)
}
```